In [23]:
from itertools import combinations
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import time


df = pd.read_csv('train_cleaned_small.csv')
df_val = pd.read_csv('val_cleaned_small.csv')

In [24]:
df = pd.read_csv('df_train_new.csv')

In [25]:
torch.manual_seed(0)

In [26]:
def clean_data(x):
    query_ids = sorted(list(set(x['query_id'])))
    ys_train = np.array(x[x['query_id']==query_ids[0]]['relevance_label'].tolist())
    ys_train_final = []
    ys_train_final.append(ys_train)
    for i in range(len(query_ids)):
        if i == 0:
            continue

        y_new = np.array(x[x['query_id']==query_ids[i]]['relevance_label'].tolist())
        ys_train_final.append(y_new)
    

    ys_train = torch.tensor(ys_train_final,dtype=torch.float32)
    
    X = np.array(x[x['query_id']==query_ids[0]].iloc[:,2:])
    x_train_final = []
    x_train_final.append(X)
    for i in range(len(query_ids)):
        if i == 0:
            continue

        x_new = np.array(x[x['query_id']==query_ids[i]].iloc[:,2:])
        x_train_final.append(x_new)
    

    X_train = torch.tensor(np.array(x_train_final),dtype=torch.float32)
    
    return X_train,ys_train
    

In [27]:
def dcg(scores):
    scores = np.array(scores,dtype = float)
    num = 2**scores-1
    for i in range(len(num)):
        num[i] /= np.log2(i+2)
    return np.sum(num)


def ndcg_k(scores, k):
    top_k = scores[:k]
    ideal_top_k = sorted(scores)[::-1][:k]
    ndcg = dcg(top_k)
    indcg = dcg(ideal_top_k)
    return ndcg/indcg

In [28]:
X_train,ys_train = clean_data(df)

In [ ]:
class ListNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, hidden_dim2, output_dim):
        super(ListNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim,hidden_dim2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

def KL_div(p, q):
    return (p * torch.log(p / q)).sum()

In [29]:
t0 = time.time()

input_dim = 136
hidden_dim = 512
hidden_dim2 = 256
output_dim = 1
learning_rate = 0.01
num_epochs = 1

model = ListNet(input_dim, hidden_dim,hidden_dim2, output_dim)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
ndcg_regret = []


for q in range(X_train.shape[0]):
    X = X_train[q]
    Y = torch.tensor(ys_train[q].reshape(50),dtype = torch.float64)
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        output = model(X)
        output = nn.functional.softmax(output, dim=0)
        
        output_tmp = np.array(output.detach().numpy())
        output_tmp = output_tmp.reshape(50)
        Y = Y.reshape(50,1)
        Y_tmp = np.array(Y)
        rank_pred = np.argsort(output_tmp)[::-1]
        rank_score = Y_tmp[rank_pred]
        ndcg = ndcg_k(rank_score, 10)
        ndcg_regret.append(ndcg)
        target = nn.functional.softmax(Y,dim=0)
        loss = KL_div(output, target).float()
        loss.backward()
        optimizer.step()

t1 = time.time()

print('Time:', t1-t0)
                        

C:\Users\chana\AppData\Local\Temp\ipykernel_12276\4087986288.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y = torch.tensor(ys_train[q].reshape(50),dtype = torch.float64)
C:\Users\chana\AppData\Local\Temp\ipykernel_12276\2468370219.py:14: RuntimeWarning: invalid value encountered in double_scalars
  return ndcg/indcg


Time: 1.7903470993041992


In [30]:
np.nanmean(ndcg_regret)

0.3894755466751973

In [31]:
n = X_train.shape[0]
ndcg_list = []
max_ndcg = 0
for i in range(n):
    output = model(X_train[i])
    output = nn.functional.softmax(output,dim = 0)
    output = np.array(output.detach().numpy())
    output = output.reshape(50)

    Y = np.array(ys_train[i].reshape(50))
    
    rank_pred = np.argsort(output)[::-1]
    rank_score = Y[rank_pred]
    ndcg = ndcg_k(rank_score, 10)
    ndcg_list.append(ndcg)
    if ndcg > max_ndcg:
        max_ndcg = ndcg
    

C:\Users\chana\AppData\Local\Temp\ipykernel_12276\2468370219.py:14: RuntimeWarning: invalid value encountered in double_scalars
  return ndcg/indcg


In [32]:
np.nanmean(ndcg_list)

0.41082175106395435

In [9]:
idx = 181
output = model(X_train[idx])
output = nn.functional.softmax(output,dim = 0)
output = np.array(output.detach().numpy())
output = output.reshape(50)

Y = np.array(ys_train[idx].reshape(50))

In [12]:
rank_pred = np.argsort(output)[::-1]
rank_score = Y[rank_pred]

In [16]:
ndcg_k(rank_score,10)

0.9740816258588203

In [17]:
X_val, ys_val = clean_data(df_val)

In [18]:
n = X_val.shape[0]
ndcg_list = []
max_ndcg = 0
for i in range(n):
    output = model(X_val[i])
    output = nn.functional.softmax(output,dim = 0)
    output = np.array(output.detach().numpy())
    output = output.reshape(50)

    Y = np.array(ys_val[i].reshape(50))
    
    rank_pred = np.argsort(output)[::-1]
    rank_score = Y[rank_pred]
    ndcg_val = ndcg_k(rank_score, 10)
    ndcg_list.append(ndcg_val)
    if ndcg_val > max_ndcg:
        max_ndcg = ndcg_val
    

0
1
3
5
10
16
23
65


In [19]:
np.nanmean(ndcg_list)

0.4316787742719746